In [19]:
!pip install requests pandas lxml

In [20]:
CLIENT_ID = "dj0yJmk9Rkgyd3NPak4yMXpNJmQ9WVdrOVZtZE5TVEJhUmprbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PTZi"
REDIRECT_URI = "https://pythonanywhere.com"
BASE_URL = "https://api.login.yahoo.com/"
SCOPE = "fspt-r"

LEAGUE_ID = "9927"
MY_TEAM_ID = "10"
OPP_TEAM_ID = "6"


In [21]:
import secrets, hashlib, base64, urllib.parse, requests

def create_pkce_pair():
    verifier = secrets.token_urlsafe(64)
    challenge = base64.urlsafe_b64encode(
        hashlib.sha256(verifier.encode()).digest()
    ).rstrip(b"=").decode()
    return verifier, challenge

code_verifier, code_challenge = create_pkce_pair()

params = {
    "client_id": CLIENT_ID,
    "redirect_uri": REDIRECT_URI,
    "response_type": "code",
    "scope": SCOPE,
    "code_challenge": code_challenge,
    "code_challenge_method": "S256",
}

auth_url = BASE_URL + "oauth2/request_auth?" + urllib.parse.urlencode(params)
print("Open this URL and approve access:\n")
print(auth_url)


Open this URL and approve access:

https://api.login.yahoo.com/oauth2/request_auth?client_id=dj0yJmk9Rkgyd3NPak4yMXpNJmQ9WVdrOVZtZE5TVEJhUmprbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PTZi&redirect_uri=https%3A%2F%2Fpythonanywhere.com&response_type=code&scope=fspt-r&code_challenge=yrMyDVtKPYd28mgzBiajVpuQx4ersvS_jcQqCz_IJqY&code_challenge_method=S256


In [22]:
code = input("Paste the 'code' value here: ").strip()

data = {
    "grant_type": "authorization_code",
    "redirect_uri": REDIRECT_URI,
    "code": code,
    "code_verifier": code_verifier,
    "client_id": CLIENT_ID
}

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "application/json",
}

response = requests.post(BASE_URL + "oauth2/get_token", data=data, headers=headers)
print("Status:", response.status_code)
print("Raw response:", response.text)

tokens = response.json()

access_token = tokens["access_token"]
refresh_token = tokens["refresh_token"]

print("Access token acquired!")


Paste the 'code' value here: g8sphbgqq5ybfrqj5geb88788c68pdcg
Status: 200
Raw response: {"access_token":"LANcekSf5QYGo69d.PiY.Wxi8ogthpVU9P4VRLz6I28etWuEgD52y6CkdbupM694PGiyZoNQo8bFYJaG0lhOh1OSrGq9FvvyhH1za7EVc3igmFUqjks35Fn2KHo6muImBklxrNizHOhIx.dY9UPBzywIEEUOHMQ60IcDExdcEXmSKyd3NGSK._Gdt9XlWkaIPln78USwX.dG49NqMNsoTipgDEZITDOu1EJNEcu0rxY1icbgarJD98BltW9MGEJ1mBiBdvlpjWqc_7cRClOXY9NEKcVoHUuv1cWwHMd06wlaRyXOdS.DONrPharVj3DrWj3IVWFB6sN2DUgZAMz9qTczVEdypB05RvEiEDpckAFWtkKcJaaU1.YHO1Uvu0cdPHwfECtdJvi_bMhez6lSo8EMYDSnlI4NLE6fbJCJdLMm3LO1VaND4RaPpeyN7_RIze53SZZrZls58PNFTC3LXXy045fOL9uk_qO_bFtvXqwQnKueqMbATcWFJV0vGRMwzIXW0FVgLzxQ0Tyg3i7kqBvAYooQr07EPBPbt8JuDG0qLmGUKXqny1V9UAPOTMcg400Apy_df7WLMQ4J1tu1WNWvX9jouCJOx9vnszvkViv_i8ZOPQtuvCFh.8gDhmq4QId61vCnhqrl4YWpLICrUvXBLPyiBebehZtJIJ2M_cGuFQT3OhuajK1YG5kGMdoUKHaJML8OeLj28a30DnHnrJbke2tPQRhlOU0h09yWdYGBcaW7m_cpQ0otcOGmr428UaDVbTFnuFkBP8Ov_xSLiYt8NqLV1QW8ZImwRNVwok3bEulMXkepOCVqyVkWz73AoiWYsRZe8lIwWhGgsYoZuqB9w8BLiDl4_ZFB_h9mpJgc9Z8Uee2h6t91uKd5a.d

In [23]:
import pandas as pd
import json

def yahoo_get(endpoint: str):
    headers = {"Authorization": f"Bearer {access_token}"}
    url = f"https://fantasysports.yahooapis.com/fantasy/v2/{endpoint}?format=json"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()

def collect_game_keys(obj, collected=None):
    """Recursively collect all 'game_key' values in a nested Yahoo JSON object."""
    if collected is None:
        collected = []

    if isinstance(obj, dict):
        if "game_key" in obj:
            collected.append(obj["game_key"])
        for v in obj.values():
            collect_game_keys(v, collected)
    elif isinstance(obj, list):
        for item in obj:
            collect_game_keys(item, collected)

    return collected

def get_latest_nba_game_key():
    j = yahoo_get("games;game_codes=nba")
    games = j.get("fantasy_content", {}).get("games", {})
    all_keys = collect_game_keys(games, [])

    if not all_keys:
        print("No game_key values found. Raw 'games' structure:")
        print(json.dumps(games, indent=2)[:2000])
        raise RuntimeError("Could not find any NBA game_key values.")

    # Yahoo uses numeric-ish game keys per season; pick the latest (max)
    # Filter to those that look like integers, just in case.
    numeric_keys = []
    for k in all_keys:
        try:
            numeric_keys.append(int(str(k)))
        except ValueError:
            pass

    if not numeric_keys:
        print("Non-numeric game_keys:", all_keys)
        raise RuntimeError("Could not interpret NBA game_keys as integers.")

    latest = max(numeric_keys)
    return str(latest)

GAME_KEY = get_latest_nba_game_key()
print("NBA Game Key (latest season):", GAME_KEY)


NBA Game Key (latest season): 466


In [24]:
import json

def find_first_dict_with_key(obj, target_key: str):
    """
    Recursively search a nested Yahoo JSON object for the first dict
    that directly contains target_key. Return that dict, or None.
    """
    if isinstance(obj, dict):
        if target_key in obj:
            return obj
        for v in obj.values():
            found = find_first_dict_with_key(v, target_key)
            if found is not None:
                return found
    elif isinstance(obj, list):
        for item in obj:
            found = find_first_dict_with_key(item, target_key)
            if found is not None:
                return found
    return None


def get_games_this_week(player_id: str) -> int:
    """
    Return the number of games this player has in the current fantasy week.
    Uses:
      - player_stats["games"] if available
      - otherwise falls back to counting entries in player_schedule
    """
    endpoint = f"player/{GAME_KEY}.p.{player_id}/stats;type=week"
    j = yahoo_get(endpoint)
    player_obj = j["fantasy_content"]["player"]

    # 1) Try player_stats.games first (simplest)
    stats_holder = find_first_dict_with_key(player_obj, "player_stats")
    if stats_holder is not None:
        ps = stats_holder["player_stats"]
        games_val = ps.get("games")
        if games_val is not None:
            try:
                return int(games_val)
            except Exception:
                pass  # fall through to schedule fallback

    # 2) Fallback: count player_schedule entries
    sched_holder = find_first_dict_with_key(player_obj, "player_schedule")
    if sched_holder is not None:
        schedule = sched_holder["player_schedule"]
        games = 0
        for k, v in schedule.items():
            if k.isdigit() and isinstance(v, dict):
                games += 1
        return games

    # 3) Nothing found – assume 0
    return 0


# Apply to your and opponent rosters
my_team["games_this_week"] = my_team["player_id"].apply(get_games_this_week)
opp_team["games_this_week"] = opp_team["player_id"].apply(get_games_this_week)

print("My team with games this week:")
display(my_team.head())
print("\nOpponent team with games this week:")
display(opp_team.head())


HTTPError: 400 Client Error: Bad Request for url: https://fantasysports.yahooapis.com/fantasy/v2/player/466.p.1295/stats;type=week?format=json